In [ ]:
# ! pip install Office365-REST-Python-Client

In [ ]:
# ! pip install azure-storage-file-datalake

In [ ]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd

In [ ]:
class SharepointToAzureBlobConnector:

    def __init__(self, username, password, sharepoint_subdomain, sharepoint_site_name, azure_account_name, azure_access_key, container_name):
        self.username = username
        self.password = password
        self.sharepoint_site_name= sharepoint_site_name
        self.url = f"https://{sharepoint_subdomain}.sharepoint.com"
        self.site = f"https://{sharepoint_subdomain}.sharepoint.com/sites/{self.sharepoint_site_name}/"
        self.azure_storage_account_name = azure_account_name
        self.azure_storage_access_key = azure_access_key
        self.container_name = container_name

    def create_sharepoint_authentication(self):
        ctx_auth = AuthenticationContext(self.site)
        try:
            if ctx_auth.acquire_token_for_user(self.username, self.password):
                CTX = ClientContext(self.site, ctx_auth)
                web = CTX.web
                CTX.load(web)
                CTX.execute_query()
                print('Authenticated into SharePoint as:', web.properties['Title'])
                return CTX
            else:
                print("Failed to authenticate.")
        except Exception as e:
            print(f"Error during SharePoint authentication:\n{e}\n{ctx_auth.get_last_error()}")

    def sharepoint_folder_details(self, folder_in_sharepoint):
        try:
            folder_names = []
            CTX = self.create_sharepoint_authentication()
            folder = CTX.web.get_folder_by_server_relative_url(folder_in_sharepoint)
            sub_folders = folder.files
            CTX.load(sub_folders)
            CTX.execute_query()
            for s_folder in sub_folders:
                folder_names.append(s_folder.properties["Name"])
            print(f"Folder details:\n{folder_names}")
        except Exception as e:
            print(f"Error retrieving folder details:\n{e}")

    def create_azure_datalake_authentication(self):
        try:
            # Initialize the DataLakeServiceClient
            service_client = DataLakeServiceClient(
                account_url=f"https://{self.azure_storage_account_name}.dfs.core.windows.net",
                credential=self.azure_storage_access_key
            )
            print("Azure Data Lake Service Client authenticated successfully.")
            return service_client
        except Exception as e:
            print(f"Error during Azure Data Lake authentication:\n{e}")

    def load_sharepoint_file_to_azure_blob(self, sharepoint_file_path: str, azure_directory_name: str):
        try:
            # Create file parameters
            file_path =    f"/sites/{self.sharepoint_site_name}/{sharepoint_file_path}"
            file_name=     file_path.split("/")[-1]
            print(f"\nfile_path:\n{file_path}")
            print(f"file_name:\n{file_name}\n")

            # Load SharePoint file
            CTX = self.create_sharepoint_authentication()
            file_response = File.open_binary(CTX, file_path)

            if file_response.status_code == 200:
                print("Status: 200,\nRequest to sharepoint was successfull!\n")
            else:
                print(f"Status: {file_response.status_code}, Request failed to sharepoint\n")

            # Authenticate Data Lake service client
            service_client = self.create_azure_datalake_authentication()

            # Create or get the file system client
            file_system_client = service_client.get_file_system_client(file_system=self.container_name)

            # Create or get the directory client
            directory_client = file_system_client.get_directory_client(azure_directory_name)

            # Create the file client in the specified directory
            file_client = directory_client.create_file(file_name)

            # Upload the content
            file_content = file_response.content
            file_client.append_data(data=file_content, offset=0, length=len(file_content))
            file_client.flush_data(len(file_content))

            print("File successfully uploaded to Azure Data Lake Storage!")
        except Exception as e:
            print(f"Error during file upload:\n{e}")

In [ ]:
# initiate the class & pass parameters
SharepointToAzureConnectorClass= SharepointToAzureBlobConnector(username= xxxxxx, 
                                                                password= xxxxxx, 
                                                                sharepoint_subdomain=xxxxxx, 
                                                                sharepoint_site_name=xxxxxx, 
                                                                azure_account_name="adlsbauapp_xxxx", 
                                                                azure_access_key=xxxxxx, 
                                                                container_name="bronze")

# Get File Name 
SharepointToAzureConnectorClass.sharepoint_folder_details(folder_in_sharepoint="Shared Documents/AzureBlobStorageConnection")

# Load file from sharepoint to Azure blob
SharepointToAzureConnectorClass.load_sharepoint_file_to_azure_blob(sharepoint_file_path= "Shared Documents/AzureBlobStorageConnection/Fleet Mgmt Master Mapping Wkst.xlsx", 
                                                                   azure_directory_name= "sharepoint")